In [1]:
import requests
import json
import re
import pandas as pd
import sklearn
import os
import nltk
import string
import newspaper

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.probability import FreqDist
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from newspaper import Article

In [2]:
########## BUILD CSV WITH LABELS, DESCRIPTION, AND CONTENT ##########

api_key = "4fe8b78ffdb64b0fb8dbfda6a3ba99f6"
url = "https://newsapi.org/v2/everything"

url_paul = ("https://newsapi.org/v2/everything?q=McCartney&domains=rollingstone.com&langauge=en&apiKey=4fe8b78ffdb64b0fb8dbfda6a3ba99f6")
url_john = ("https://newsapi.org/v2/everything?q=Lennon&domains=rollingstone.com&langauge=en&apiKey=4fe8b78ffdb64b0fb8dbfda6a3ba99f6")

params_paul = {"q": "Paul McCartney",
               "language": "en",
               "sortBy": "publishedAt",
               "apiKey": api_key}

params_john = {"q": "John Lennon",
               "language": "en",
               "sortBy": "publishedAt",
               "apiKey": api_key}

response_paul = requests.get(url_paul)
response_john = requests.get(url_john)

if response_paul.status_code == 200:
    json_txt_paul = response_paul.json()
    paul_articles = json_txt_paul.get("articles", [])
    df_paul = pd.DataFrame(paul_articles)

else:
    print(f"Error: {response_paul.status_code}, {response_paul.text}")

df_paul['full_text'] = df_paul['url'].apply(lambda url: (lambda a: (a.download(), a.parse(), a.text)[2])(Article(url)))
df_paul['label'] = len(df_paul)*['paul']

if response_john.status_code == 200:
    json_txt_john = response_john.json()
    john_articles = json_txt_john.get("articles", [])
    df_john = pd.DataFrame(john_articles)

else:
    print(f"Error: {response_john.status_code}, {response_john.text}")

df_john['full_text'] = df_john['url'].apply(lambda url: (lambda a: (a.download(), a.parse(), a.text)[2])(Article(url)))
df_john['label'] = len(df_john)*['john']

df_paul_john = pd.concat([df_paul, df_john])

df_paul_john.to_csv("paul_john_articles.csv", index = True)